# Measurement of Liquid Column Height by Hydrostatic Pressure Difference

## Principle

The method determines the height $h$ of a liquid column (for example, liquid xenon) inside a **vertical suction line** by comparing inlet–outlet pressures measured at a known **mass flow rate** $\dot{m}$.

A separate **gas-only calibration** establishes the pressure-squared gradient

$$
G(\dot{m}) = \frac{p_\text{in}^2 - p_\text{out}^2}{L}
$$

as a function of the mass flow $\dot{m}$.

During liquid operation, the additional **hydrostatic head** of the liquid phase increases the measured pressure drop; solving the balance yields the liquid height.

---

## Step 1 – Gas-only calibration

1. Configure the suction line (length $L$, inner diameter $D$).
2. Circulate **gas only** (no liquid present).
3. Measure the absolute inlet and outlet pressures $p_{\text{in},0}$, $p_{\text{out},0}$ for several mass flows $\dot{m}_k$.
4. Compute

   $$
   G_k = \frac{p_{\text{in},0k}^2 - p_{\text{out},0k}^2}{L}.
   $$

5. Fit a simple power law

   $$
   G(\dot{m}) = c\,\dot{m}^{\,n},
   $$

   or record the tabulated pairs $(\dot{m}_k, G_k)$ for interpolation.

This calibration implicitly contains all gas-side friction and minor-loss effects and removes the need to know gas properties.

---

## Step 2 – Liquid run at matched flow

1. Operate the system at the **same mass flow** as used in calibration  
   (or use the fitted $G(\dot{m})$ to correct if the flow differs slightly).
2. Measure the inlet and outlet pressures $p_{\text{in},1}$ and $p_{\text{out},1}$.
3. Assume:
   - The lower portion of the tube (length $h = L_\ell$) is filled with liquid.
   - The upper portion $L_g = L - L_\ell$ contains gas.
   - Flow is steady and vertical.

---

## Step 3 – Governing equation

The pressure balance for the two-phase column is

$$
\big[p_{\text{in},1} - (\alpha + \gamma)\,h\big]^2 - p_{\text{out},1}^2
= G(\dot{m})\,\big(L - h\big),
$$

where

$$
\begin{aligned}
\gamma &= \rho_\ell\,g
&&\text{(hydrostatic gradient, Pa/m)},\\[6pt]
\alpha &= \frac{2 f_\ell}{D\,\rho_\ell\,A^2}\,\dot{m}^{\,2}
&&\text{(liquid friction term, optional)},\\[6pt]
A &= \frac{\pi D^2}{4},\\[6pt]
G(\dot{m}) &= \frac{p_{\text{in},0}^2 - p_{\text{out},0}^2}{L}
&&\text{(from calibration)}.
\end{aligned}
$$

Neglecting $\alpha$ is usually acceptable for liquid xenon because the hydrostatic term dominates.

---

## Step 4 – Solve for the liquid height

Expanding gives a quadratic in $h$:

$$
a\,h^2 + b\,h + c = 0,
$$

with

$$
\begin{aligned}
a &= (\alpha + \gamma)^2,\\[4pt]
b &= -\,2(\alpha + \gamma)\,p_{\text{in},1} + G(\dot{m}),\\[4pt]
c &= p_{\text{in},1}^2 - p_{\text{out},1}^2 - G(\dot{m})\,L.
\end{aligned}
$$

The physical solution $0 \le h \le L$ is

$$
h = \frac{-b + \sqrt{\,b^2 - 4ac\,}}{2a}.
$$

---

## Step 5 – Verification and averaging

Repeat measurements for several mass flows $\dot{m}_j$ (using the same calibration law $G(\dot{m})$).  
If the liquid level is fixed by an evaporator or heat exchanger, all inferred heights $h_j$ should coincide within uncertainty.  
Average (or take the median) of the consistent $h_j$ values to obtain the final column height.

---

## Advantages of the method

- Matched-flow operation cancels most gas-side uncertainties.  
- The calibration curve $G(\dot{m})$ allows later operation at different flows without repeating gas-only measurements.  
- Only simple quantities are needed: pressures, mass flow, geometry, and liquid density.  
- The quadratic form provides an analytic solution for $h$.  
- The hydrostatic term dominates, so small deviations in $f_\ell$ or $G(\dot{m})$ have minimal impact.

In [ ]:
import math
import numpy as np
from dataclasses import dataclass

# ------------------------------
# Utilities & physics
# ------------------------------

def area(D): 
    return math.pi*D*D/4.0

def G_from_pressures(pin, pout, L):
    """
    pin, pout in Pa(abs); L in m
    Returns G = (pin^2 - pout^2) / L  [Pa^2/m]
    """
    pin = np.asarray(pin); pout = np.asarray(pout)
    return (pin**2 - pout**2) / L

@dataclass
class GasCalibration:
    """Log-log power-law calibration: G(mdot) = c * mdot^n  [Pa^2/m]"""
    c: float
    n: float

    def G(self, mdot):
        mdot = np.asarray(mdot, dtype=float)
        return self.c * (mdot**self.n)

def fit_gas_calibration(mdot, pin, pout, L):
    """
    Fit G(mdot) = c * mdot^n from gas-only measurements at (mdot_k, pin_k, pout_k).
    mdot: array of kg/s, pin/pout: Pa(abs), L: m
    Returns GasCalibration(c, n) and a dict of fit diagnostics.
    """
    mdot = np.asarray(mdot, dtype=float)
    Gk   = G_from_pressures(pin, pout, L).astype(float)

    # Guard against nonpositive values for log-fit
    if np.any(mdot <= 0) or np.any(Gk <= 0):
        raise ValueError("All mdot and G_k must be > 0 for log-log fit.")

    # log-log linear regression
    X = np.log(mdot)
    Y = np.log(Gk)
    n, logc = np.polyfit(X, Y, 1)
    c = float(np.exp(logc))

    # diagnostics
    Yhat = n*X + logc
    resid = Y - Yhat
    rmse_log = float(np.sqrt(np.mean(resid**2)))
    r2 = float(1.0 - np.var(resid, ddof=1)/np.var(Y, ddof=1))

    return GasCalibration(c=c, n=float(n)), {
        "rmse_log": rmse_log,           # RMS error in log-space
        "r2_log": r2,                   # coefficient of determination in log-space
        "G_meas": Gk, 
        "G_fit": np.exp(Yhat)
    }

def liquid_height_vertical(
    pin1, pout1, mdot1, L, D, rho_l, 
    calibration: GasCalibration,
    f_l=None,                # Darcy friction factor for liquid (optional)
    g=9.81
):
    """
    Compute liquid height h (vertical suction) from one liquid run.

    Inputs:
      pin1, pout1 : Pa(abs)
      mdot1       : kg/s
      L           : tube length [m]
      D           : inner diameter [m]
      rho_l       : liquid density [kg/m^3]
      calibration : GasCalibration object (from gas-only fit)
      f_l         : (optional) Darcy friction factor for liquid segment. If None, friction neglected.
      g           : 9.81 m/s^2 by default

    Returns:
      dict with h [m], and diagnostics (a,b,c,disc,alpha,gamma,S).
    """
    A = area(D)
    gamma = rho_l * g                  # Pa/m (hydrostatic slope)
    # Gas-side slope at this flow from calibration:
    S = calibration.G(mdot1)           # Pa^2/m

    # Optional liquid friction per meter (Pa/m):
    if f_l is None:
        alpha = 0.0
    else:
        # alpha = (2 f_l / (D * rho_l * A^2)) * mdot1^2
        alpha = (2.0 * f_l / (D * rho_l * A * A)) * (mdot1**2)

    # Quadratic coefficients for: a h^2 + b h + c = 0
    a = (alpha + gamma)**2
    b = -2.0*(alpha + gamma)*pin1 + S
    c = pin1**2 - pout1**2 - S*L

    disc = b*b - 4.0*a*c
    if disc < 0:
        raise ValueError(f"Negative discriminant: b^2-4ac = {disc:.3e}. "
                         "Check inputs or calibration.")

    # Two roots; pick the one in [0, L]
    h1 = (-b + math.sqrt(disc)) / (2.0*a)
    h2 = (-b - math.sqrt(disc)) / (2.0*a)
    candidates = [h for h in (h1, h2) if 0.0 <= h <= L]
    if not candidates:
        raise ValueError(f"No physical root in [0, {L}] (roots: {h1:.6g}, {h2:.6g}).")

    # Heuristic pick: choose the root closer to gas-only estimate
    est = L * (1.0 - (pin1**2 - pout1**2)/max(S*L, 1e-30))
    h = min(candidates, key=lambda x: abs(x - est))

    return {
        "h_m": float(h),
        "a": a, "b": b, "c": c, "disc": disc,
        "alpha_Pa_per_m": alpha, "gamma_Pa_per_m": gamma, "S_Pa2_per_m": float(S)
    }

def liquid_heights_multi(
    pins, pouts, mdots, L, D, rho_l, calibration: GasCalibration, f_l=None, g=9.81
):
    """
    Vectorized helper: compute heights for multiple liquid runs.
    Returns list of dicts and a robust aggregate (median ± MAD-based sigma).
    """
    pins  = np.asarray(pins,  dtype=float)
    pouts = np.asarray(pouts, dtype=float)
    mdots = np.asarray(mdots, dtype=float)

    results = []
    hs = []
    for pin1, pout1, md in zip(pins, pouts, mdots):
        r = liquid_height_vertical(pin1, pout1, md, L, D, rho_l, calibration, f_l=f_l, g=g)
        results.append(r)
        hs.append(r["h_m"])

    hs = np.array(hs, dtype=float)
    h_med = float(np.median(hs))
    h_spread = float(1.4826 * np.median(np.abs(hs - h_med)))  # robust sigma estimate

    return results, h_med, h_spread

# ------------------------------
# Example usage (fill with your data)
# ------------------------------

if __name__ == "__main__":
    # --- GAS-ONLY CALIBRATION (example numbers; REPLACE with your measurements) ---
    # Arrays of mdot [kg/s], pin0 [Pa], pout0 [Pa] at the same tube length L
    L = 1.20               # m (tube length)
    D = 4.6e-3             # m (tube ID)
    mdot_g = np.array([0.0008, 0.0012, 0.0018])    # kg/s
    pin0   = np.array([1.35e5, 1.45e5, 1.60e5])    # Pa abs
    pout0  = np.array([1.05e5, 1.05e5, 1.05e5])    # Pa abs

    calib, diag = fit_gas_calibration(mdot_g, pin0, pout0, L)
    print("Calibration: G(m) = c * m^n  with  c=%.3e  n=%.3f" % (calib.c, calib.n))
    print("Log-space R^2 =", diag["r2_log"])

    # --- LIQUID RUN(S) (example numbers; REPLACE with your measurements) ---
    rho_l = 2900.0         # kg/m^3  (LXe ~ 2.9 g/cc at ~-100 C)
    # optional: include liquid friction (Darcy f_l), else set None
    f_l = None              # e.g. 0.02  (if you’ve computed it), None to neglect

    # Single run:
    pin1, pout1, md1 = 1.80e5, 1.20e5, 0.0015   # Pa, Pa, kg/s
    res = liquid_height_vertical(pin1, pout1, md1, L, D, rho_l, calib, f_l=f_l)
    print("Single run height h =", res["h_m"], "m")

    # Multiple runs:
    pins  = [1.75e5, 1.85e5, 1.95e5]
    pouts = [1.20e5, 1.20e5, 1.20e5]
    mdots = [0.0012, 0.0015, 0.0018]
    results, h_med, h_sig = liquid_heights_multi(pins, pouts, mdots, L, D, rho_l, calib, f_l=f_l)
    print("Per-run heights:", [round(r["h_m"], 4) for r in results])
    print("Aggregate (median ± robust sigma):", round(h_med,4), "±", round(h_sig,4), "m")